# CMDfit
Author: Seth Gossage  
Version: 0.1
***
Below are descriptions of the data, model, and statistical analysis (likelihood function, priors, posterior calculation) relevant to this code. Currently the rotation parameter $\Omega$ is **not** included as a free parameter, but this will be implemented in the future.

## Data
In my project, the data come from observations of invidual stars (e.g. $N$ such observations). We often delineate where a star is in its evolution by examining how bright and hot it is at the time of observation. Each data point is a measurement of the flux from a star, which is recorded as an apparent magnitude (apparent because this is how bright the star appears to us on Earth, and not how intrinsically bright it is). Thus,

* My data points are apparent magnitudes, which I will label as $m_{ij}$. The subscripting demarcates the magnitude taken in the $j^{th}$ wavelength filter (e.g. the magnitude in either the U, B, V, R, or I filter) and corresponding to the $i^{th}$ data point (so the magnitude recorded for one of the $N$ observations).  

* Each data point has recorded uncertainties arising from photometric uncertainty, i.e. Poisson random noise (or photon shot noise) as a result of the randomness of photon counting. I will call these $\sigma_{ij}$, and the subscripting works the same as above for the magnitudes.
***

## Model
The expected data are magnitude measurements of individual stars; to model these observations, model stars evolved with the MESA code (Paxton 2011, 2012, 2013) are the basis for the models currently in use. These are stars given various initial conditions and allowed to evolve. A separate code is used to transform these models into a simulated observable stellar population by making use of various photometric tables. This process accounts for things such as extinction and the pecularities of the various optics that may have been used to acquire a data set.

The stellar populations of stellar clusters are expected to be coeval, and this is the sort of population that this code is designed to analyze. To this end, the MESA models are used to create isochrones -- i.e. collections of stellar models all at a single age. An isochrone theoretically matches the description of a coeval stellar population, and so determining the parameters that yield a particular isochrone is a way to estimate the parameters of such populations.

We match models to observations and so the primary way of interfacing the two is to look at the magnitudes produced by our models to draw comparisons.

* My model points are the magnitudes calculated from these stellar models. I will use $\mu_{ij}$ to symbolize them; the subscripting serves the same purpose as for the data points: $j$ to mark which filter I am considering and $i$ for the $i^{th}$ star's magnitude in that filter.
  
Given a set of these isochrones (each at a different age, metallicity, and intial rotation rate), we have a range of ages and other parameters to test with and find which isochrone best simulates the data.
***

## The Likelihood Function $p(D|\theta,M,I)$  

The likelihood function is the probability of a model successfully simulating a dataset. I am going to follow a similar framework to what is used in the paper by van Dyk et al., 2009 (http://arxiv.org/pdf/0905.2547.pdf).  

There are several complicating factors that will need to be considered, but as a preliminary setup, I can form a likelihood based on the following points.

* At a given time in a star's life it is expected to have a certain magnitude. (At later stages this is not true generally, star's pulsate dramatically and can go through periods of being relatively dim, bright, and then dim again. We are fitting isochrones primarily to the star's in earlier stages of their life, so this is not a major issue in this project.)  

* The uncertainties $\sigma_{ij}$ are photometric uncertainties which result from photon shot noise (or Poisson noise) during the observation process.

* In this project we examine star's in a stellar cluster (the Hyades specifically) and so all stars are assumed to be at roughly the same distance, age, and share common initial rotation rates and metallicities. (This is based on the idea that stars in clusters form from a single cloud of dust, at roughly similar ages (within a number of years), and are also relatively close to each other as a consequence of this.)

* Additionally, extinction (magnitude shifts due to intervening dust between us and the cluster, which scatters light) is assumed to affect all stars in the cluster in the same way, so shifts due to it are irrelevant, as long as stars are in the same cluster. (This is mostly important to factor in when calculating magnitudes from the stellar evolution models.)

Therefore, matching measured magnitudes (data) to predicted magnitudes (model), a preliminary likelihood function may be written as:  

$$p(D | \theta, M, I) = \Pi^N_{i=1}\left(\Pi^n_{j=1}\left[\frac{1}{\sqrt{2\pi\sigma_{ij}^2}}\exp\left\{{\frac{-(m_{ij} - \mu_{ij})}{2\sigma_{ij}}}\right\}^2\right]\right)$$  

Combining individual likelihoods which are checking the probabilities between the $i^{th}$ predicted magnitude and its corresponding measured magnitude in each individual $j^{th}$ filter (so how far the measurement deviates from the prediction, considering uncertainties). So, I am also saying above that there could be as many as $n$ filters being considered in the likelihood.

I have calculated magnitudes available from stellar models, and recorded magnitudes, along with their uncertainties available from data tables. So this likelihood is calcuable.

### Complications  

The comparison between a given data and model point above is not as straight forward as I have described so far. For several reasons, the magnitude given by a data point could inherently violate the model. Here are the known reasons:

* The observed single star may actaully be two stars, i.e. a binary system. In this case comparing the magnitude of a binary system to the magnitude of a single star as assumed in the model calculations would be an inappropriate comparison.

* The observed star could actually be a "field star", rather than a member of the stellar cluster. Field stars are stars that just happen to have magnitudes that make them look like plausible candidates of the cluster, when in reality they are much brighter stars located much further away, or fainter stars located closer to us.  

Dealing with these complications can be done in the following ways: 

** Binary Systems  
(discussed in http://iopscience.iop.org/article/10.1088/0004-637X/696/1/12/pdf (DeGennaro et al, 2009), Sect. 2.2)
**
***
We can treat each computed magnitude from an isochrone as if it were a binary system. This does not change the likelihood written above, but it will change how we sample the prior involving the initial mass parameter for star (or now it should be called "system") $i$. Details on this are below, in the section on the selected priors.  
Including binaries does change the meaning of $\mu_{ij}$, however, as now the model "star" has a magnitude that is a combination of the magnitudes of both stars in the system. I.e.  

$$\mu_{ij} = -2.5log_{10}\left[10^{-\mu_{ij,1}/2.5} + 10^{-\mu_{ij,2}/2.5} \right]$$  

where  

$$\mu_{ij,1} = \mu_{j,1}(M_{i1},\Theta)$$  

and

$$\mu_{ij,2} = \mu_{j,2}(M_{i2},\Theta)$$

are the model magnitudes for the primary and secondary components of the binary system respectively; they are functions of the initial mass of either the primary or secondary star ($M_{i1}$ or $M_{i2}$) and the remaining, cluster-wide parameters for e.g. age and metallicity store in the parameter set $\Theta$. Again, this does not affect the form of the likelihood function, but under the hood this is the effect of including binaries in this treatment.  

Binary systems are expected to be fairly common in the universe (as many as 50% of all observed stars may be binary systems), and so including this possibility in our formalism is not too unreasonable.


** Field Stars  
(discussed in http://iopscience.iop.org/article/10.1088/0004-637X/696/1/12/pdf (DeGennaro et al, 2009), Sect. 2.3
and http://arxiv.org/pdf/0905.2547.pdf (van Dyk, 2009), Sect. 3.3)  
**
***
This can be handled by adapting the likelihood function to a mixture model (i.e. a likelihood which considers the data as a mixture of cluster *and* field stars). So I will have a $P_{field}$, which is the probability that a data point is a field star (or the amplitude of the field star distrubution in this mixture). This modifies the likelihood a bit in the following way:  

$$p(D | \theta, M, I) = \Pi^N_{i=1}\left(\Pi^n_{j=1}\left[\frac{\left(1-P_{field}\right)}{\sqrt{2\pi\sigma_{ij}^2}}\exp\left\{{\frac{-(m_{ij} - \mu_{ij})}{2\sigma_{ij}}}\right\}^2\right]\right) + P_{field}\mathcal{L}_{field}$$  

(this is similar to the van Dyk paper's equation 5, but they define the probability concerning cluster membership in the reverse way)  where $\mathcal{L}_{field}$ is the contribution to the likelihood function coming from field stars. The van Dyk paper speaks about possible ways to treat this, and they use a uniform distribution defined as  

$$\mathcal{L}_{field} = \frac{1}{\Pi_{j=1}^n \left(max_j - min_j\right)}$$  

Here, $max_j$ is the maximum magnitude value in filter $j$, and $min_j$ is the minimum magnitude in this filter. I will adopt this for the time being, but it looks like improvements could be made here possibly.

*******


Thus, the likelihood function has the freedom to consider data points as binary systems and as field stars in determing the best-fit parameters. Essentially these are additions that give the this function tools to deal with potential outliers, and these tools are founded in astrophysical reasoning. However, as noted above improvements to how these factors are treated could possibly be made.

***
# Priors
The parameters being considered are rotation $\Omega / \Omega_{crit}$ (or just $\Omega$), metallicity (Z or [Fe/H]), age, and initial mass. Since there is a potential for data points to represent binary systems, the initial mass prior is not as straight forward as it could be. Nevertheless, here are the descriptions:

For $\Omega$ and age (actually log10 of the age), a uniform prior of the form  

$$\frac{1}{\theta_{max} - \theta_{min}}$$

($\theta$ being a particular parameter, such as age, and the $max/min$ subscripts denoting the max or min of the parameter range being sampled from) will be used.

**Handling of the Initial Mass**
We would like to consider the possibility that a data point may be a binary system, rather than a single star. Therefore (following deGennaro, 2009), we will assume that every data point is a binary system; the catch is that we allow for the mass of the secondary star to become zero (in which case the data point is not a binary system). This, the initial mass prior is split into a prior for the secondary star's intial mass, and another prior for the primary star's initial mass.  

The primary star will be assumed to be the more massive star, and its prior is taken as a Gaussian distribution with variance and mean set by the Miller & Scalo (1979) initial mass function (IMF). This may be represented as a half-Gaussian with the log10 of the primary star's initial mass:

$$p(M_1 | \theta, D, I) \propto \exp\left(-\frac{1}{2}\left(\frac{log(M_1) + 1.02}{0.677}\right)^2\right)$$

What is left then is just how to handle the secondary mass. For this, a uniform prior in the range of 0 to the primary star's mass value is used.

**Handling of [Fe/H]**
The metallcity may be constrined a little more as well. It is commonly believed that the metallicity in the solar neighborhood is near solar (most of the material around us is similar in composition to the Sun). This idea may be expressed in placing a Gaussian prior on [Fe/H], peaked at the solar value [Fe/H] = $0.0$ and having a standard deviation of $\pm 0.1$ dex.  

$$p([Fe/H]|\theta, D, I) = 22.5\exp\left(-\frac{1}{2}\left(\frac{[Fe/H]}{0.1}\right)^2\right)$$

***

# MCMC Method

This code uses the `emcee` package and in particular its `EnsembleSampler()` routine in order to sample the parameter space. This is an affine-invariant sampler which involves the use of walkers which are distributed randomly in parameter space and are able to communicate with each other. This enhances the standard Metropolis-Hastings method in allowing an ensemble of walkers to explore different regions and call each other over to regions of high probability, significantly increasing the sampling process in some cases.

The priors and likelihood function detailed above serve as guides for the MCMC algorithm in determining what is and what is not likely when comparing a model to a data point.

See the Tutorial.ipynb notebook for a demonstration of the code and to see how this method performs in this code.
